In [ ]:
# Garbage dumped here :)


# from trainer import Trainer
# from datasets import load_dataset

# dataset = load_dataset("imdb", split="train")

# train = Trainer()

# test_dataset = load_dataset("imdb", split="test")

# prompt = "Hey how are you doing?"
# trans_tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
# mamba_tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
# trans_model_inputs = trans_tokenizer(prompt, return_tensors="pt").to('cuda')
# mamba_inputs = mamba_tokenizer(prompt, return_tensors= "pt")

# print(mamba_inputs)

# from transformers import AutoTokenizer, AutoModelForCausalLM

# trans_model = AutoModelForCausalLM.from_pretrained(
#     'EleutherAI/gpt-neo-125M',
#     torch_dtype="auto",
#     device_map="cuda"
# )

# from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
# mamba_model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf")


In [ ]:
# ! pip install transformers datasets evaluate

In [1]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
from tqdm import tqdm
from hybrid_model import HybridModel, HybridModelTextClassification
from datasets import load_dataset
from transformers import MambaModel

/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @c

In [2]:
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Load tokenizer and pretrained model
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
##tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")  # Replace with the actual Mamba model ID

# Load IMDb dataset

dataset = load_dataset("imdb", split="train")

In [3]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

In [38]:
with torch.cuda.device(device):  
        info = torch.cuda.mem_get_info()
        print(info[0]/(1024 ** 3), info[1]/(1024 ** 3))

10.59246826171875 10.74658203125


In [36]:
for i in range(10):
    device = f'cuda:{i}' if torch.cuda.is_available() else 'cpu'
    if device == 'cpu':
        break
    with torch.cuda.device(device):  
        info = torch.cuda.mem_get_info()
        print(info[0]/(1024 ** 3), info[1]/(1024 ** 3))


0.21160888671875 10.74658203125
10.59246826171875 10.74658203125
10.59246826171875 10.74658203125
10.59246826171875 10.74658203125
10.59246826171875 10.74658203125
0.03778076171875 10.74658203125
10.59246826171875 10.74658203125
10.59246826171875 10.74658203125


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [4]:
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    temp = tokenizer([item["text"] for item in examples], padding=True, truncation=True, return_tensors="pt")
    out = {}
    out["input_ids"] = temp["input_ids"].to(device)
    out["attention_mask"] = temp["attention_mask"].to(device)
    out["labels"] = torch.tensor([item["label"] for item in examples], device=device)
    return out

In [5]:
dataloader = DataLoader(dataset, batch_size=4, collate_fn=preprocess_function, shuffle = True)

In [6]:
from transformers import AutoModel
transformer_backbone= AutoModel.from_pretrained('EleutherAI/gpt-neo-125M')

In [7]:
mamba_backbone = MambaModel.from_pretrained('state-spaces/mamba-130m-hf')

In [8]:
model = HybridModelTextClassification(transformer_backbone, mamba_backbone, 2).to(device)
model.hybrid_model.mamba_model.to("cuda:2")
model.hybrid_model.splitters.to("cuda:3")
model.hybrid_model.combiners.to("cuda:3")
model.train()

for param in model.hybrid_model.trans_model.parameters():
    param.requires_grad = False

for param in model.hybrid_model.mamba_model.parameters():
    param.requires_grad = False

trainer_params = []
for combiner in model.hybrid_model.combiners.parameters():
    trainer_params.append(combiner)

for spliter in model.hybrid_model.splitters.parameters():
    trainer_params.append(spliter)

In [9]:
epochs = 3
lr=1e-3
weight_decay = 0.01

In [10]:
optimizer = torch.optim.AdamW(params=trainer_params, lr=lr, weight_decay=weight_decay)

In [11]:
total_loss = 0
num_batches = 0
for epoch in range(epochs):
  for idx, batch in enumerate(dataloader):
    # input_ids = batch["input_ids"].to(device)
    # attention_mask = batch["attention_mask"].to(device)
    loss = model(**batch).loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # acc = accuracy_metric.compute(predictions=logits.argmax(dim=-1), references=batch["labels"])['accuracy']

    total_loss += loss.item()
    num_batches += 1
    if idx % 10 == 0:
      print("epoch:", epoch, " ---- iteration:", idx, " ---- loss:", total_loss/num_batches)
      total_loss = 0
      num_batches = 0

epoch: 0  ---- iteration: 0  ---- loss: 146.41368103027344
epoch: 0  ---- iteration: 10  ---- loss: 1049.5060059547425
epoch: 0  ---- iteration: 20  ---- loss: 1411.42265625
epoch: 0  ---- iteration: 30  ---- loss: 2772.8953491210937
epoch: 0  ---- iteration: 40  ---- loss: 2037.297720399499
epoch: 0  ---- iteration: 50  ---- loss: 1135.7773742675781
epoch: 0  ---- iteration: 60  ---- loss: 909.5135223388672
epoch: 0  ---- iteration: 70  ---- loss: 200.20476055145264
epoch: 0  ---- iteration: 80  ---- loss: 342.7853576660156
epoch: 0  ---- iteration: 90  ---- loss: 370.9487907409668
epoch: 0  ---- iteration: 100  ---- loss: 324.2792191505432
epoch: 0  ---- iteration: 110  ---- loss: 90.60240926742554
epoch: 0  ---- iteration: 120  ---- loss: 108.87160077095032
epoch: 0  ---- iteration: 130  ---- loss: 138.71552772521972
epoch: 0  ---- iteration: 140  ---- loss: 40.97762036919594


OutOfMemoryError: CUDA out of memory. Tried to allocate 406.00 MiB. GPU 1 has a total capacity of 10.75 GiB of which 24.69 MiB is free. Including non-PyTorch memory, this process has 10.72 GiB memory in use. Of the allocated memory 10.08 GiB is allocated by PyTorch, and 462.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [32]:
import inspect
arg_names = inspect.getfullargspec(method).args
print(arg_names)

['self', 'hidden_states', 'layer_past', 'attention_mask', 'head_mask', 'use_cache', 'output_attentions', 'cache_position']


In [34]:
input_ids = batch["input_ids"]
attention_mask = batch["attention_mask"]
labels = batch["labels"]

In [36]:
from projector import Combiner, Splitter

In [41]:
dim1 = transformer_backbone.wte.weight.shape[-1]
dim2 = mamba_backbone.embeddings.weight.shape[-1]

# Create intermediate layers and LM head
combiners = torch.nn.ModuleList([Combiner(dim1, dim2) for _ in range(12)]).to(device)
splitters = torch.nn.ModuleList([Splitter(dim1, dim2) for _ in range(12)]).to(device)
proj_dim = max(dim1, dim2)

In [44]:
trans_layers = transformer_backbone.h
mamba_layers = mamba_backbone.layers

In [ ]:
trans_t_emb = transformer_backbone.wte(input_ids)
trans_p_emb = transformer_backbone.wpe(torch.tensor([[i for i in range(input_ids.shape[1])]]).to(input_ids.device))
trans_input_emb = trans_t_emb + trans_p_emb

# Pass the input through each block and intermediate layers
combined_emb = trans_input_emb  # (batch_size, seq_len, proj_dim)
hidden_states = (combined_emb, )

trans_layers_per_block = len(transformer_backbone.h) // 12
mamba_layers_per_block = len(mamba_backbone.layers) // 12
for i in range(12):
    trans_input_emb, mamba_input_embeds = splitters[i](combined_emb)
    for j in range(trans_layers_per_block):
        trans_input_emb = trans_layers[trans_layers_per_block * i + j](trans_input_emb, attention_mask=attention_mask)[0]    
    for k in range(mamba_layers_per_block):
        mamba_input_embeds = mamba_layers[mamba_layers_per_block * i + k](mamba_input_embeds, attention_mask=attention_mask)
    
    combined_emb = combiners[i](trans_input_emb, mamba_input_embeds)
    hidden_states += (combined_emb, )

In [42]:
trans_t_emb = transformer_backbone.wte(input_ids)
trans_p_emb = transformer_backbone.wpe(torch.tensor([[i for i in range(input_ids.shape[1])]]).to(input_ids.device))
trans_input_emb = trans_t_emb + trans_p_emb

# Pass the input through each block and intermediate layers
combined_emb = trans_input_emb  # (batch_size, seq_len, proj_dim)
hidden_states = (combined_emb, )

trans_layers_per_block = len(transformer_backbone.h) // 12
mamba_layers_per_block = len(mamba_backbone.layers) // 12

In [55]:
trans_input_emb, mamba_input_embeds = splitters[0](combined_emb)

In [45]:
trans_input_emb = trans_layers[0](trans_input_emb, attention_mask=attention_mask)[0] 

IndexError: too many indices for tensor of dimension 2

In [48]:
trans_layers[0].forward(trans_input_emb, cache_position=attention_mask)

(tensor([[[ 40.2624, -29.5371,   9.1647,  ...,   2.5638,  -3.5764,   3.1672],
          [ 48.7600,  -3.0616,   0.2260,  ...,  -1.6918, -26.6392, -17.1404],
          [ 38.2042,  -5.0733, -10.2056,  ...,  -6.8845, -30.4292,  -0.4299],
          ...,
          [ 25.7060,  -3.4444,  -8.5557,  ...,  10.1624, -29.4412,  -4.7316],
          [ 26.7100,  -3.0962,  -9.2371,  ...,  10.4994, -29.5563,  -4.7530],
          [ 27.1860,  -3.3466,  -9.6219,  ...,  10.8257, -30.6160,  -4.7499]],
 
         [[ 44.9628, -17.4808,   0.2680,  ...,  12.5463, -13.4202,   8.9818],
          [ 33.4319, -10.9950,  -2.5597,  ..., -10.4071, -22.1687, -11.2520],
          [ 32.2356,  -8.4184, -12.7499,  ..., -15.0528, -30.6298,   5.6062],
          ...,
          [ 43.5172,   0.2967,  -4.0872,  ...,  10.3203, -26.8207, -14.8821],
          [ 41.2498,   0.4535,  -2.7486,  ...,   8.2166, -27.0470, -15.4812],
          [ 39.9374,  -0.7058,   0.2018,  ...,   5.9308, -26.8740, -13.8932]],
 
         [[ 43.2288, -24.223

In [52]:
method = mamba_layers[0].forward

In [53]:
import inspect
arg_names = inspect.getfullargspec(method).args
print(arg_names)

['self', 'hidden_states', 'cache_params', 'cache_position', 'attention_mask']
